In [ ]:
import os
import astropy.coordinates as coord
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.io import fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import binned_statistic_2d

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from thriftshop.potentials import potentials

coord.galactocentric_frame_defaults.set('v4.0');

In [ ]:
t = at.Table.read('../data/apogee-parent-sample.fits')
t = t[t['GAIA_PARALLAX'] > 0.5]
len(t)

In [ ]:
c = coord.SkyCoord(ra=t['RA']*u.deg,
                   dec=t['DEC']*u.deg,
                   distance=1000 / t['GAIA_PARALLAX'] * u.pc,
                   pm_ra_cosdec=t['GAIA_PMRA']*u.mas/u.yr,
                   pm_dec=t['GAIA_PMDEC']*u.mas/u.yr,
                   radial_velocity=t['VHELIO_AVG']*u.km/u.s)
galcen = c.transform_to(coord.Galactocentric)

In [ ]:
z = galcen.z.to_value(u.kpc)
vz = galcen.v_z.to_value(u.km/u.s)
elem = t['MG_FE']

In [ ]:
zlim = 2 # kpc
vlim = 100. # km/s
vstep = 4
zstep = 75 / 1e3
vzz_bins = (np.arange(-vlim, vlim+1e-3, vstep),
            np.arange(-zlim, zlim+1e-3, zstep))

fig, axes = plt.subplots(1, 2, figsize=(12, 5),
                         constrained_layout=True)

stat = binned_statistic_2d(vz, z, elem, statistic='mean',
                           bins=vzz_bins)

vmin, vmax = np.percentile(elem, [15, 85])

ax = axes[0]
cs = ax.pcolormesh(stat.x_edge, stat.y_edge, stat.statistic.T, 
                   cmap='cividis', vmin=vmin, vmax=vmax)
cb = fig.colorbar(cs, ax=ax, aspect=40)

ax.set_xlabel('v_z')
ax.set_ylabel('z')

ax = axes[1]
H, *_ = np.histogram2d(vz, z, bins=vzz_bins)
cs = ax.pcolormesh(stat.x_edge, stat.y_edge, H.T, 
                   cmap='cividis', 
                   norm=mpl.colors.LogNorm(1, 3e2))
cb = fig.colorbar(cs, ax=ax, aspect=40)

ax.set_xlabel('v_z')
# ax.set_ylabel('z')

fig.set_facecolor('w')

# ---

fig, ax = plt.subplots(figsize=(6, 5))

stat = binned_statistic_2d(vz, z, elem, statistic='mean',
                           bins=vzz_bins)

vmin, vmax = np.percentile(elem, [15, 85])

HH = stat.statistic.copy()
HH[H < 3] = np.nan
cs = ax.pcolormesh(stat.x_edge, stat.y_edge, 
                   HH.T, cmap='cividis', vmin=vmin, vmax=vmax)
cb = fig.colorbar(cs, ax=ax, aspect=40)

ax.set_xlabel('v_z')
ax.set_ylabel('z')
fig.tight_layout()

In [ ]:
w0s = gd.PhaseSpacePosition(pos=([[-8.122, 0, 0],
                                  [-8.122, 0, 0]]*u.kpc).T,
                            vel=([[12.9, 245.6, 25.],
                                  [12.9, 245.6, 50.]]*u.km/u.s).T)

orbits = {}
for k, p in potentials.items():
    orbits[k] = p.integrate_orbit(w0s, dt=1*u.Myr, t1=0, t2=6*u.Gyr)

In [ ]:
sorted_keys = sorted(
    orbits.keys(), 
    key=lambda k: potentials[k]['disk'].parameters['m'])

In [ ]:
plot_zlim = 1.75
plot_vzlim = 100

# -----
# vz, z
fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for k, ax in zip(sorted_keys, axes):
    _ = orbits[k].plot(['v_z', 'z'], axes=[ax], 
                       auto_aspect=False, units=[u.km/u.s, u.kpc])
    try:
        ax.set_title(f'${float(k):.1f}' + r' \, {\rm M}_{\rm disk}$')
    except ValueError:
        ax.set_title(f'{k}')

axes[1].set_ylabel('')
axes[2].set_ylabel('')

axes[0].set_xlim(-plot_vzlim, plot_vzlim)
axes[0].set_ylim(-plot_zlim, plot_zlim)

fig.tight_layout()

# -----
# R, z

fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for k, ax in zip(sorted_keys, axes):
    _ = orbits[k].cylindrical.plot(
        ['rho', 'z'], axes=[ax], 
        auto_aspect=False, units=[u.kpc, u.kpc])
    
    try:
        ax.set_title(f'${float(k):.1f}' + r' \, {\rm M}_{\rm disk}$')
    except ValueError:
        ax.set_title(f'{k}')

axes[1].set_ylabel('')
axes[2].set_ylabel('')

axes[0].set_xlim(7.5, 11.5)
axes[0].set_ylim(-plot_zlim, plot_zlim)

fig.tight_layout()

In [ ]:
zlim = 1.75 # kpc
vlim = 75. # pc/Myr
vstep = 4.
zstep = 75 / 1e3
vzz_bins = (np.arange(-vlim, vlim+1e-3, vstep),
            np.arange(-zlim, zlim+1e-3, zstep))

In [ ]:
import itertools

quad_vz = np.array([])
quad_z = np.array([])
quad_elem = np.array([])
for i, j in itertools.product([-1,1], [-1, 1]):
    quad_vz = np.concatenate((quad_vz, i * vz))
    quad_z = np.concatenate((quad_z, j * z))
    quad_elem = np.concatenate((quad_elem, elem))

quad_stat = binned_statistic_2d(quad_vz, quad_z, 
                                quad_elem, statistic='mean',
                                bins=vzz_bins)
stat = binned_statistic_2d(vz, z, 
                           elem, statistic='mean',
                           bins=vzz_bins)

quad_counts, *_ = np.histogram2d(quad_vz, quad_z, bins=vzz_bins)
counts, *_ = np.histogram2d(vz, z, bins=vzz_bins)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(17, 5), 
                         constrained_layout=True,
                         sharex=True, sharey=True)

H = quad_stat.statistic
# H = convolve(quad_stat.statistic, Gaussian2DKernel(1.))
H[quad_counts <= 1] = np.nan

# vmin, vmax = np.percentile(elem, [5, 95])
# vmin, vmax = (-0.4, 0.2)
vmin, vmax = (-0.02, 0.1)

for ax in axes:
    cs = ax.pcolormesh(stat.x_edge, stat.y_edge, 
                       H.T, cmap='magma', vmin=vmin, vmax=vmax)
    ax.set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')

cb = fig.colorbar(cs, ax=axes, aspect=40)
# cb.set_label(r'$[{\rm Fe} / {\rm H}]$')
cb.set_label(r'$[{\rm Mg} / {\rm Fe}]$')

for k, color, ax in zip(sorted_keys,
                        ['w', 'w', 'w'],
                        axes):
    o = orbits[k]
    o_z = o.z.to_value(u.kpc)
    o_vz = o.v_z.to_value(u.km/u.s)
    ax.plot(o_vz, o_z, marker='', color=color, alpha=0.5)
    
    try:
        ax.set_title(f'${float(k):.1f}' + r' \, {\rm M}_{\rm disk}$')
    except ValueError:
        ax.set_title(f'{k}')
    
axes[0].set_xlim(-vlim, vlim)
axes[0].set_ylim(-zlim, zlim)
    
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

fig.suptitle(r'$v_{\rm circ} = 229\,{\rm km}\,{\rm s}^{-1}$ for all potentials',
             fontsize=24)

fig.set_facecolor('w')

In [ ]:
for o, p in zip([orbit_wrong_lo, orbit_fidu, orbit_wrong_hi],
                [mw_wrong_lo, mw_fidu, mw_wrong_hi]):
    o_long = p.integrate_orbit(w0s, dt=2*u.Myr, 
                               t1=0, t2=100 * o.estimate_period().max(),
                               Integrator=gi.DOPRI853Integrator)
    
    print(gd.find_actions(o_long[:, 0], N_max=8)['actions'])
#     print(gd.find_actions(o_long[:, 0], N_max=8)['angles'])